<a href="https://colab.research.google.com/github/NaufalYogaPratama/analisis-sentimen-MLBB/blob/main/Analisis_Sentimen_Mobile_Legends_NYP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Analisis Sentimen mobile Legends: Bang Bang**

## Install Library

In [7]:
!pip install google-play-scraper pandas tqdm
!pip install Sastrawi
print("Instalasi selesai!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.1 MB/s eta 0:00:00
Instalasi selesai!


## Import Library

In [8]:
import pandas as pd
import numpy as np
from google_play_scraper import Sort, reviews
from tqdm import tqdm
import logging
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tqdm import tqdm

## Scraping Process

In [4]:
# Setup logging dasar
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Konfigurasi Scraping ---
# ID Aplikasi Mobile Legend: Bang Bang di Google Play Store
APP_ID = 'com.mobile.legends'
# Jumlah ulasan yang ingin kita kumpulkan
REVIEW_COUNT = 12000 # Kita ambil lebih untuk cadangan jika ada data duplikat/kosong
# Nama file output
OUTPUT_FILENAME = 'mobile_legends_reviews.csv'

logging.info(f"Memulai scraping untuk aplikasi: {APP_ID}")

# Melakukan scraping ulasan
# Library ini akan menangani paginasi secara otomatis untuk mengambil jumlah yang kita minta
result, _ = reviews(
    APP_ID,
    lang='id',           # Bahasa: Indonesia
    country='id',        # Negara: Indonesia
    sort=Sort.NEWEST,    # Urutkan berdasarkan yang paling baru
    count=REVIEW_COUNT,
    filter_score_with=None # Ambil semua skor dari 1 sampai 5
)

if not result:
    logging.warning("Tidak ada ulasan yang berhasil di-scrape.")
else:
    # Konversi hasil ke DataFrame pandas
    df_reviews = pd.DataFrame(result)

    # Pilih hanya kolom yang relevan untuk proyek kita
    df_final = df_reviews[['content', 'score', 'at']].copy()
    df_final.rename(columns={'content': 'review_text', 'score': 'rating', 'at': 'review_date'}, inplace=True)

    # Membersihkan data dari baris yang tidak memiliki teks ulasan
    df_final.dropna(subset=['review_text'], inplace=True)

    # Menghapus ulasan yang duplikat
    df_final.drop_duplicates(subset=['review_text'], keep='first', inplace=True)

    logging.info(f"Berhasil mengumpulkan {len(df_final)} ulasan unik.")

    # Menyimpan DataFrame ke file CSV di lingkungan Colab
    df_final.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8')

    logging.info(f"Dataset berhasil disimpan sebagai '{OUTPUT_FILENAME}'.")

    # Menampilkan 5 baris pertama dari dataset
    print("\nBerikut adalah 5 sampel data pertama:")
    print(df_final.head())


Berikut adalah 5 sampel data pertama:
                                         review_text  rating  \
0  maaf kk semua player sekalu afk terus dan saya...       1   
1                           DARK SYSTEM SEMUA ISINYA       1   
2                           kurangin dark sistem nya       5   
3  apalah ini game jaringan bagus, udah pake wifi...       1   
4  nontonn ajing buat meta line banyak drak siste...       1   

          review_date  
0 2025-06-25 17:20:47  
1 2025-06-25 17:20:44  
2 2025-06-25 17:20:31  
3 2025-06-25 17:20:21  
4 2025-06-25 17:19:11  


## Pra-pemrosesan Data dan Labeling

In [6]:
# Memuat dataset yang sudah di-scrape sebelumnya
try:
    df = pd.read_csv('mobile_legends_reviews.csv')
    print("Dataset berhasil dimuat!")
    print(f"Jumlah data: {len(df)} ulasan.")
except FileNotFoundError:
    print("File 'mobile_legends_reviews.csv' tidak ditemukan. Pastikan Anda sudah menjalankan cell scraping.")

# Membuat fungsi untuk melabeli sentimen berdasarkan rating
def label_sentiment(rating):
    if rating in [1, 2]:
        return 'Negatif'
    elif rating == 3:
        return 'Netral'
    elif rating in [4, 5]:
        return 'Positif'
    return None # Jika ada rating di luar 1-5

# Terapkan fungsi ke kolom 'rating' untuk membuat kolom 'sentiment'
df['sentiment'] = df['rating'].apply(label_sentiment)

# Memeriksa hasil pelabelan dan distribusi data
print("\nDistribusi data berdasarkan sentimen:")
print(df['sentiment'].value_counts())

# Menampilkan 5 sampel data dengan kolom baru
print("\nBerikut adalah 5 sampel data dengan label sentimen:")
print(df.head())

Dataset berhasil dimuat!
Jumlah data: 11220 ulasan.

Distribusi data berdasarkan sentimen:
sentiment
Negatif    7333
Positif    3271
Netral      616
Name: count, dtype: int64

Berikut adalah 5 sampel data dengan label sentimen:
                                         review_text  rating  \
0  maaf kk semua player sekalu afk terus dan saya...       1   
1                           DARK SYSTEM SEMUA ISINYA       1   
2                           kurangin dark sistem nya       5   
3  apalah ini game jaringan bagus, udah pake wifi...       1   
4  nontonn ajing buat meta line banyak drak siste...       1   

           review_date sentiment  
0  2025-06-25 17:20:47   Negatif  
1  2025-06-25 17:20:44   Negatif  
2  2025-06-25 17:20:31   Positif  
3  2025-06-25 17:20:21   Negatif  
4  2025-06-25 17:19:11   Negatif  


## Text Cleaning

In [9]:
# Inisialisasi tqdm untuk integrasi dengan pandas
tqdm.pandas()

# --- 1. Case Folding ---
def case_folding(text):
    return text.lower()

# --- 2. Normalisasi Kata & Pembersihan Karakter ---
# Membuat dictionary untuk normalisasi kata-kata slang/jargon
normalization_dict = {
    'gk': 'tidak', 'ga': 'tidak', 'gak': 'tidak', 'nggak': 'tidak',
    'jg': 'juga', 'jgn': 'jangan', 'sdh': 'sudah',
    'yg': 'yang', 'dg': 'dengan', 'tp': 'tapi',
    'bgt': 'banget', 'utk': 'untuk',
    'afk': 'away from keyboard', 'mabar': 'main bareng', 'lag': 'lambat',
    'op': 'overpowered', 'gg': 'good game', 'plis': 'please',
    'hp': 'handphone', 'mntp': 'mantap', 'keren': 'bagus'
    # Tambahkan kata lain jika ditemukan saat eksplorasi data
}

def normalize_and_clean(text):
    # Hapus URL
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Hapus karakter non-alphanumeric (kecuali spasi)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Normalisasi kata
    words = text.split()
    normalized_words = [normalization_dict.get(word, word) for word in words]
    return ' '.join(normalized_words)

# --- 3. Stopword Removal ---
# Membuat stopword remover
stopword_factory = StopWordRemoverFactory()
# Menambahkan beberapa stopword yang mungkin relevan dengan konteks game
more_stopwords = ['moonton', 'mobile', 'legend', 'legends', 'bang']
data_stopwords = stopword_factory.get_stop_words() + more_stopwords
remover = stopword_factory.create_stop_word_remover()

def remove_stopwords(text):
    return remover.remove(text)

# --- 4. Stemming ---
# Membuat stemmer
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

def stem_text(text):
    return stemmer.stem(text)

# --- Gabungkan semua fungsi menjadi satu pipeline ---
def preprocess_text(text):
    text = case_folding(text)
    text = normalize_and_clean(text)
    text = remove_stopwords(text)
    text = stem_text(text)
    return text

# --- Terapkan pipeline ke kolom 'review_text' ---
# Proses ini mungkin memakan waktu beberapa menit. Tqdm akan menunjukkan progress bar.
print("Memulai proses text cleaning...")
df['cleaned_review'] = df['review_text'].progress_apply(preprocess_text)
print("Proses text cleaning selesai.")

# Menampilkan hasil
print("\nContoh hasil pembersihan teks:")
print(df[['review_text', 'cleaned_review']].head())

Memulai proses text cleaning...


100%|██████████| 11220/11220 [25:46<00:00,  7.26it/s]

Proses text cleaning selesai.

Contoh hasil pembersihan teks:
                                         review_text  \
0  maaf kk semua player sekalu afk terus dan saya...   
1                           DARK SYSTEM SEMUA ISINYA   
2                           kurangin dark sistem nya   
3  apalah ini game jaringan bagus, udah pake wifi...   
4  nontonn ajing buat meta line banyak drak siste...   

                                      cleaned_review  
0  maaf kk semua player sekalu away from keyboard...  
1                              dark system semua isi  
2                             rangin dark sistem nya  
3  apa game jaring bagus udah pake wifi tetep aja...  
4  nontonn ajing buat meta line banyak drak siste...  
